In [1]:
!pip install -q -U keras-tuner

/var/folders/4k/lxd3n8m934z_82zsdzv73d5r0000gn/T/ipykernel_11682/3452742943.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [11]:
import pandas as pd
import numpy as np
import warnings
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
import IPython
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
warnings.filterwarnings(action='ignore')

In [3]:
df = pd.read_csv("data/modified_train_0417.csv")
x_train = df.drop(columns=['loan_status', 'addr_state', 'loan_amnt_intallment_rate'])
y_train = df['loan_status']

In [4]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [5]:
x_train.shape

(1091767, 93)

In [6]:
# minmax scaler 찾아보기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [7]:
y_train=to_categorical(y_train, 2).astype(int)
y_validation=to_categorical(y_validation, 2).astype(int)
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# y_train = y_train.values
# y_train = encoder.fit_transform(y_train.reshape(-1, 1))

In [8]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [23]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)
hp = kt.HyperParameters()

def build_model(hp):
    model = keras.Sequential()
    hp_units = hp.Int('units', min_value = 10, max_value = 90, step = 32)
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(10))
    model.add(Dense(2, activation='softmax'))
    optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model
tuner = kt.Hyperband(hypermodel = build_model, objective = 'val_accuracy', max_epochs = 10, directory = 'my_dir')
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_validation, y_validation), callbacks = [ClearTrainingOutput()])

Trial 3 Complete [00h 00m 33s]
val_accuracy: 0.798391580581665

Best val_accuracy So Far: 0.7986159920692444
Total elapsed time: 00h 01m 41s


In [27]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 10.



In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs = 10, validation_data = (x_validation, y_validation))

In [11]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')